<img src="https://img.icons8.com/bubbles/100/000000/3d-glasses.png" style="height:50px;display:inline"> University of Haifa - Computer Vision course


## Homework 2 - 3D Vision
---

### <a style='color:red'> Due Date: Thursday 12.6.2025 </a>

### <img src="https://img.icons8.com/bubbles/50/000000/upload-to-cloud.png" style="height:50px;display:inline"> Submission Guidelines
---
#### READ THIS CAREFULLY
* Submission only in **pairs**, unless you have already got an approval to work on your own

* Submission is only through Moodle

* You can choose your working environment:
    * `Jupyter Notebook`, locally with <a href="https://www.anaconda.com/distribution/">Anaconda</a> or online on <a href="https://colab.research.google.com/">Google Colab</a>
        * Colab also supports running code on GPU, but you will not need it for this excercise.
    * Python IDE such as <a href="https://www.jetbrains.com/pycharm/">PyCharm</a> or <a href="https://code.visualstudio.com/">Visual Studio Code</a>.
        * Both allow editing and running Jupyter Notebooks.
<div style="margin-top:6px;"></div>

* You should submit only this ipynb notebook file, **with all cells executed such that the results will be included**.
<div style="margin-top:6px;"></div>

* Read the instructions carefully:
    * You have functions to complete and scripts to write - look for <span style="color:red">''' YOUR CODE HERE '''</span>
        * Make sure to properly document your code
        * Notice where you can use existing implementations (e.g. opencv) freely and in which cases you have to provide your own implementation of certain steps
    * You have some explanations and discussions to write - look for <span style="color: red; font-weight: bold;">YOUR EXPLANATION\DISCUSSION HERE</span>
        * Make sure to answer shortly but informatively
<div style="margin-top:6px;"></div>

* Code of Honor:
    * We wish to avoid dealing with copied homework. However, we will not hesitate to take serious actions against students that are caught with violations of the Honor Code. Note that in this course, it is rather easy to detect similar submissions, as most of them require your "personal" touch.     
<div style="margin-top:6px;"></div>

* Submission date:
    * Submission date of 12.6.2025 is final and will not be delayed
    * However, you may submit up to 2 days late (till Sunday 14.6.2025) at the cost of 4 points per day
    * Any (fully justified and documented) requests for delay must be sent till Sunday 8.6 at the latest
    * The excercise is long and isn't easy - **make sure to start early**


### <img src="https://img.icons8.com/dusk/64/000000/python.png" style="height:50px;display:inline"> Python Libraries
---

* `numpy`
* `matplotlib`
* `opencv` (or `scikit-image`)
* `scikit-learn`
* `scipy`
* Anything else you need (`os`, `pandas`, `csv`, `json`,...)

# <font color="red">IMPORTANT:</font> Make sure that each output image has a title to it.

# Part 1 - Constraining putative matches using geometry
---

### Implement the functions below (separately), given a pair of images to be matched:

## 1. Obtain SIFT-based matches
* Compute SIFT features in each of the images
* Perform SIFT matching between the two sets of features, to obtain a set of matches M

## 2. Visualize a set of matches
* Visualize the image pair, side-by-side, with thin lines connecting matching points (You may visualize a large random subset, if there are too many matches and the result is too cluttered)

## 3. Recover the Epipolar geometry
* Given the set of matches, estimate a Fundamental matrix F

## 4. Visualize the Epipolar geometry
* Given F, sample 10 2D points, uniformly at random, in image 1 and compute (using F), the epipolar lines in image 2.
* Visualize the image pair, side-by-side. Choose 10 unique different colors (preferably spread uniformly over some colorspace) and then use colored markers to overlay the points in image 1 and the corresponding colors to overlay the epipolar lines in image 2.
* Repeat the above, by switching roles (random points in image 2 and matching epipoles in image 1).

## 5. Constrain a set of matches, by filtering them with the obtained geometry
* Find and return the subset M' of the original set of matches M, that 'agree' with the obtained F, up to some error threshold.


### Using the above functions, for each pair of images in data/pairs_to_match, perform the following:
* Visualize a simple SIFT-based set of matches
* Estimate the fundamental matrix F and visualize the epipolar geometry
* Constrain the set of matches using F and visualize the result.
* Do you observe that constraining the matches to the epipolar geometry (given by F) is helpful in any way? (Was it able to filter out all of the outliers? Did it filter out any inliers?). Explain in the second cell below.



In [ ]:
# ALL OF YOUR CODE FOR PART 1 SHOULD BE IN THIS CELL

# REQUIRED OUTPUTS (For each of the input stereo pairs in data/pairs_to_match):
# 1. Visualize matches
# 2. Visualize epipolar geometry
# 3. Visualize matches after constraints

"""
YOUR CODE HERE
"""

<span style="color: red; font-weight: bold;">YOUR EXPLANATION HERE</span>

# Part 2 - Middle-View synthesis from a rectified stereo pair.
---

### In this section, given a *rectified* image pair, we wish to generate/synthesize the image that would have been captured by a camera situated exactly in the middle, between the pair of poses.

## We will follow two different strategies to do so:
* Strategy A: Via photometric stereo
* Strategy B: Via sparse reconstruction

## Details for Strategy A:
* Compute a (dense) disparity map between the pair of images: **Implement your own (don't use a ready library function for this)**
* Use this map to synthesize the 'central' image
* Explain what problems arise in this approach and how your solution tries to deal with them

## Details for Strategy B:
* Use the code from the previous parts to obtain a set of matches - ones that obeys with the geometry. How would you do this even without knowing the fundamental matrix?
* Using triangulation, find the locations of the interest points in the new 'central' image (In practice - this can be done by simply using the disparity for each match). At this point - You should have 'triplet' matches, each consisting of 1 point in each of the 3 viewpoints.
* Take the set of (match) points in one of the images (left, right or middle - to your choice) and sub-divide the image into small triangular or quadrilateral sub-regions, whose vertices are placed on the matched interest points. This can be done, for example, by computing any kind of triangulation over the interest points (see the 'Dylan' example under the 'Examples' folder). Once obtaining the subdivision (e.g. triangulation), it can be transfered to the other two images, using the corresponding matching points.
* For each region (triangle or quadrilateral), copy the contents from one of the input images to the target (middle) image, using either an affine transformation (for a triangle) or a homography (for a quadrilateral).
* Explain what problems arise in this approach and how your solution tries to deal with them.

## Implement your solutions and expain them in the following 4 cells.


In [ ]:
# ALL OF YOUR CODE FOR STATEGY A SHOULD BE IN THIS CELL

# REQUIRED OUTPUTS (For each of the input stereo pairs in data/rectified_pairs):
# 1. Visualize, side-by-side: left-image | right-image
# 2. Visualize, side-by-side: left-disparity-image | right_disparity-image
# 3. Visualize, by constructing an infinite-loop GIF of the four images: img_1->img_mid->img_2->img_mid->,
#    using the following given function:

import imageio
from IPython.display import Image

def create_and_display_gif(image_files, gif_path='animation.gif', duration=0.5):
    """
    Creates a GIF from a list of image files and displays it in a Jupyter notebook.

    Parameters:
    - image_files (list of str): Paths to the input image files.
    - gif_path (str): Path where the output GIF will be saved.
    - duration (float): Duration per frame in seconds.

    Returns:
    - IPython.display.Image object displaying the GIF.
    """
    with imageio.get_writer(gif_path, mode='I', duration=duration) as writer:
        for filename in image_files:
            image = imageio.imread(filename)
            writer.append_data(image)

    return Image(filename=gif_path)

# example usage:
# image_files = ['frame_1.jpg', 'frame_middle.jpg', 'frame_2.jpg', 'frame_middle.jpg'] 
# create_and_display_gif(image_files, gif_path='my_gif.gif', duration=0.5)

"""
YOUR CODE HERE
"""

### Explanation of your solution (short, in bullets) and interpretation of results of strategy A:

<span style="color: red; font-weight: bold;">YOUR EXPLANATION HERE</span>

In [ ]:
# ALL OF YOU CODE FOR STATEGY B SHOULD BE IN THIS CELL

# REQUIRED OUTPUTS (For each of the input stereo pairs in data/rectified_pairs):
# 1. Visualize, side-by-side: left-image | right-image (both with match-points overlaid on them - using small markers)
# 2. Visualize, side-by-side: left-image | white-image | right image (with overlayed corresponding subdivisions)
# 3. Visualize, by constructing an infinite-loop GIF of the four images: img_1->img_mid->img_2->img_mid->
 
"""
YOUR CODE HERE
"""

### Explanation of your solution (short, in bullets) and interpretation of results of strategy B:

<span style="color: red; font-weight: bold;">YOUR EXPLANATION HERE</span>

# Part 3 - Motion Segmentation
---

### In this section, you are given a pair of images, taken from two different viewpoints, at different times. The captured scene is known to be static, except for a single rigid (non-flexible) object that has moved between the two  captures. Your goal is to 'segment' (classify) the set of matches between the images, as belonging to either 'static' (background) or 'dynamic' (moving object) or 'outliers' (inconsistent).

## Instructions
* Use only the functions that you had implemented in part 1
* Show results for the image pair under the 'data/pairs_to_segment' folder
* Capture yourselves two additional image pairs, save them in that same folder, and produce results for them too.


## Required Visualization (for each image pair):
-- See the 'segmentation' solution outputs under the 'examples' folder
* Show the image pair, with 'outlier' matches overlaid (show only a random subset if there are too many)
* Show the image pair, with 'static' matches overlaid (show only a random subset if there are too many)
* Show the image pair, with 'dynamic' matches overlaid (show only a random subset if there are too many

## Implement your solution in the next cell and explain it in short in the following one.


In [ ]:
# ALL OF YOUR CODE FOR PART 3 SHOULD BE IN THIS CELL

# REQUIRED OUTPUTS (For each of the input stereo pairs in data/pairs_to_match):
# 1. Visualize matches
# 2. Visualize epipolar geometry
# 3. Visualize matches after constraints

"""
YOUR CODE HERE
"""

### Explanation of your solution (short, in bullets) and interpretation of your results.

<span style="color: red; font-weight: bold;">YOUR EXPLANATION HERE</span>